<a href="https://colab.research.google.com/github/maryblue31/JavaBlue/blob/master/N_grams_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1st implementation


import math
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk.tokenize import TweetTokenizer
from collections import Counter
from nltk.util import ngrams
from google.colab import files
uploaded = files.upload()



#read text from file
def read_file(file):
    f = open(file, "r")
    text = f.read()
    return text

text = read_file("train_data.txt")
text = text.lower()

#split text into sentences
sentences = sent_tokenize(text)
sentences_tokenized = [];

#tokenize sentences
tweet_wt = TweetTokenizer()

for sent in sentences:
    sent_tok = tweet_wt.tokenize(sent);
    sentences_tokenized.append(sent_tok);

#tokenize text 
tokens = tweet_wt.tokenize(text);

#find word frequencies
count = nltk.FreqDist(tokens)





#set vocabulary (distinct words with frequency >10)
vocabulary = set()
for key in tokens:
    if(tokens.count(key)>=10):
        vocabulary.add(key);

vocab_size = len(vocabulary)+1 #1 is the unknown token






#replace the unknown words in the training set with the token unknown(replace only in sentences_tokenized)
for sent in sentences_tokenized:
    for i in range(len(sent)):
        if(sent[i] not in vocabulary):
            sent[i] = 'UNK'




#hash tables with ngrams frequencies
unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()

for sent in sentences_tokenized:
    unigram_counter.update([gram for gram in ngrams(sent, 1, pad_left=True, pad_right=True,left_pad_symbol='<s>',right_pad_symbol='<e>') ])
    bigram_counter.update([gram for gram in ngrams(sent,2,pad_left=True, pad_right=True,left_pad_symbol='<s>', right_pad_symbol='<e>')])
    trigram_counter.update([gram for gram in ngrams(sent,3,pad_left=True, pad_right=True,left_pad_symbol='<s>', right_pad_symbol='<e>')])










# cross entropy and perplexity

alpha = 0.01;


test_text = read_file("test_data.txt")
test_text = test_text.lower()

sentences_of_test = sent_tokenize(test_text)
sentences_tokenized_of_test = [];

for sent in sentences_of_test:
    sent_tok = tweet_wt.tokenize(sent);
    sentences_tokenized_of_test.append(sent_tok);

for sent in sentences_tokenized_of_test:
    for i in range(len(sent)):
        if(sent[i] not in vocabulary):
            sent[i] = 'UNK'

#for bigram model
sum_prob = 0
bigram_cnt = 0

for sent in sentences_tokenized_of_test:
    sent = ['<s>'] + sent + ['<e>']

    # Iterate over the bigrams of the sentence
    for idx in range(2, len(sent)):
        bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] + alpha) / (unigram_counter[(sent[idx-1],)] + alpha*vocab_size)
        sum_prob += math.log2(bigram_prob)
        bigram_cnt += 1

HC = -sum_prob / bigram_cnt
perpl = math.pow(2,HC)
print("Cross Entropy of bigram model: {0:.3f}".format(HC))
print("Perplexity of bigram model: {0:.3f}".format(perpl))

#for trigram model
sum_prob = 0
trigram_cnt = 0

for sent in sentences_tokenized_of_test:
    sent = ['<s>'] + ['<s>'] + sent + ['<e>'] + ['<e>']

    # Iterate over the trigrams of the sentence
    for idx in range(4,len(sent) - 1):
        trigram_prob = (trigram_counter[(sent[idx-2],sent[idx-1], sent[idx])] + alpha) / (bigram_counter[(sent[idx-2],sent[idx-1])] + alpha*vocab_size)
        sum_prob += math.log2(trigram_prob)
        trigram_cnt+=1

HC = -sum_prob / trigram_cnt
perpl = math.pow(2,HC)
print("Cross Entropy of trigram model: {0:.3f}".format(HC))
print("Perplexity of trigram model: {0:.3f}".format(perpl))


# Beam Search


#find bigram probability
def findBigramPropability(start, end):
    bigram_prop = (bigram_counter[(start, end)] + alpha) / (unigram_counter[(start,)] + alpha*vocab_size)
    bigram_prop = math.log2(bigram_prop);
    return bigram_prop




    




# algorithm Levenshtein

def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        return func(*args, **kwargs)
    helper.calls = 0
    helper.__name__= func.__name__

    return helper
memo = {}
@call_counter
def levenshtein(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    cost = 0 if s[-1] == t[-1] else 1
       
    i1 = (s[:-1], t)
    if not i1 in memo:
        memo[i1] = levenshtein(*i1)
    i2 = (s, t[:-1])
    if not i2 in memo:
        memo[i2] = levenshtein(*i2)
    i3 = (s[:-1], t[:-1])
    if not i3 in memo:
        memo[i3] = levenshtein(*i3)
    res = min([memo[i1]+1, memo[i2]+1, memo[i3]+cost])
    
    return res






# Dictionary with nearest words

def min_dist_word(token, beamwidth = 10 ):
    mydict = {}
    for i in range(0,beamwidth):
        mydict[i] = 500 

    for word in vocabulary:
        d = levenshtein(token,word)
        key_max = max(mydict.keys(), key=(lambda k: mydict[k]))
        
        if d < mydict[key_max]:
            mydict.pop(key_max)
            mydict[word] = d
           
            

    for key in mydict:
        mydict[key] = math.log2(1/(mydict[key] + 1))

    return mydict # returns for each token, one dictionary with { word : 1/(dist +1) }



# Spelling corrector    

def findPath(sequence,beamwidth = 10):
    l1 = 0.4
    l2 = 0.6

    tokens = tweet_wt.tokenize(sequence);
    
    tokens = ['<s>'] + tokens + ['<e>']

    path1 = [('<s>',1)]
    path2 = [('<s>',1)]

    for i in range(1,len(tokens)-1):
        
        next_word_dict = min_dist_word(tokens[i], beamwidth)
        
        list = []
        for word in next_word_dict:
            
            
            p = l2*next_word_dict[word] + l1*findBigramPropability(path1[i-1][0], word )
           
            word_prob = [p,word,'path1']
            list.append(word_prob)

            p = l2*next_word_dict[word] + l1* findBigramPropability(path2[i-1][0],word )
            
            word_prob = [p,word,'path2']
            list.append(word_prob)

        w1 = list.index(max(list))
        w1 = list.pop(w1)
        word1 = w1[1]
      
        

        w2 = list.index(max(list))
        w2 = list.pop(w2)
        word2 = w2[1]

        

        if (w1[2] == 'path1' and w2[2] == 'path1'):
          path2 = path1
          path1.append([word1, w1[0]])
          path2.append([word2, w2[0]])
        elif (w1[2] == 'path2' and w2[2] == 'path2'):
          path1 = path2
          path1.append([word1, w1[0]])
          path2.append([word2, w2[0]])
        elif (w1[2] == 'path1' and w2[2] == 'path2' ):
          path1.append([word1,w1[0]])
          path2.append([word2,w2[0]])
        else:
          path1.append([word2, w2[0]])
          path2.append([word1, w1[0]])



        if i == len(tokens) - 2 :    # when we reach the end of the sentence
            sum1 = 0
            sum2 = 0

            

            for i in range(len(path1)):
                sum1 += path1[i][1]
            

            for i in range(len(path2)):
                sum2 += path2[i][1]
            

            if (sum1 > sum2):
                string = ''
                for i in range(1,len(path1)):
                    string = string + path1[i][0] + ' '
                return string

            else:
                string = ''
                for i in range(1,len(path2)):
                    string = string + path2[i][0] + ' '
                return string



sentence = input('Give a sentence')   #test with input sentence
print(findPath(sentence))




In [ ]:
#2nd implementation


import math
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk.tokenize import TweetTokenizer
from collections import Counter
from nltk.util import ngrams
import re
from google.colab import files
uploaded = files.upload()


#read text from file
def read_file(file):
    f = open(file, "r")
    text = f.read()
    return text

text = read_file("train_data.txt");
text = text.lower()

#split text into sentences
sentences = sent_tokenize(text)
sentences_tokenized = [];

#tokenize sentences
tweet_wt = TweetTokenizer()

for sent in sentences:
    sent_tok = tweet_wt.tokenize(sent);
    sentences_tokenized.append(sent_tok);

#tokenize text 
tokens = tweet_wt.tokenize(text);

#find word frequencies
count = nltk.FreqDist(tokens)





#set vocabulary (distinct words with frequency >10)
vocabulary = set()
for key in tokens:
    if(tokens.count(key)>=10):
        vocabulary.add(key);

vocab_size = len(vocabulary)+1 #1 is the unknown token






#replace the unknown words in the training set with the token unknown(replace only in sentences_tokenized)
for sent in sentences_tokenized:
    for i in range(len(sent)):
        if(sent[i] not in vocabulary):
            sent[i] = 'UNK'




#hash tables with ngrams frequencies
unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()

for sent in sentences_tokenized:
    unigram_counter.update([gram for gram in ngrams(sent, 1, pad_left=True, pad_right=True,left_pad_symbol='<s>',right_pad_symbol='<e>') ])
    bigram_counter.update([gram for gram in ngrams(sent,2,pad_left=True, pad_right=True,left_pad_symbol='<s>', right_pad_symbol='<e>')])
    trigram_counter.update([gram for gram in ngrams(sent,3,pad_left=True, pad_right=True,left_pad_symbol='<s>', right_pad_symbol='<e>')])










# cross entropy and perplexity

alpha = 0.01;


test_text = read_file("test_data.txt")
test_text = test_text.lower()

sentences_of_test = sent_tokenize(test_text)
sentences_tokenized_of_test = [];

for sent in sentences_of_test:
    sent_tok = tweet_wt.tokenize(sent);
    sentences_tokenized_of_test.append(sent_tok);

for sent in sentences_tokenized_of_test:
    for i in range(len(sent)):
        if(sent[i] not in vocabulary):
            sent[i] = 'UNK'

#for bigram model
sum_prob = 0
bigram_cnt = 0

for sent in sentences_tokenized_of_test:
    sent = ['<s>'] + sent + ['<e>']

    # Iterate over the bigrmas of the sentence
    for idx in range(2, len(sent)):
        bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] + alpha) / (unigram_counter[(sent[idx-1],)] + alpha*vocab_size)
        sum_prob += math.log2(bigram_prob)
        bigram_cnt += 1

HC = -sum_prob / bigram_cnt
perpl = math.pow(2,HC)
print("Cross Entropy of bigram model: {0:.3f}".format(HC))
print("Perplexity of bigram model: {0:.3f}".format(perpl))

#for trigram model
sum_prob = 0
trigram_cnt = 0

for sent in sentences_tokenized_of_test:
    sent = ['<s>'] + ['<s>'] + sent + ['<e>'] + ['<e>']

    # Iterate over the trigrams of the sentence
    for idx in range(4,len(sent) - 1):
        trigram_prob = (trigram_counter[(sent[idx-2],sent[idx-1], sent[idx])] + alpha) / (bigram_counter[(sent[idx-2],sent[idx-1])] + alpha*vocab_size)
        sum_prob += math.log2(trigram_prob)
        trigram_cnt+=1

HC = -sum_prob / trigram_cnt
perpl = math.pow(2,HC)
print("Cross Entropy of trigram model: {0:.3f}".format(HC))
print("Perplexity of trigram model: {0:.3f}".format(perpl))



# Beam Search

#find bigram probability
def findBigramPropability(start, end):
    bigram_prop = (bigram_counter[(start, end)] + alpha) / (unigram_counter[(start,)] + alpha*vocab_size)
    bigram_prop = math.log2(bigram_prop);
    return bigram_prop




    




# algorithm Levenshtein 

def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        return func(*args, **kwargs)
    helper.calls = 0
    helper.__name__= func.__name__

    return helper
memo = {}
@call_counter
def levenshtein(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    cost = 0 if s[-1] == t[-1] else 1
       
    i1 = (s[:-1], t)
    if not i1 in memo:
        memo[i1] = levenshtein(*i1)
    i2 = (s, t[:-1])
    if not i2 in memo:
        memo[i2] = levenshtein(*i2)
    i3 = (s[:-1], t[:-1])
    if not i3 in memo:
        memo[i3] = levenshtein(*i3)
    res = min([memo[i1]+1, memo[i2]+1, memo[i3]+cost])
    
    return res





# Dictionary with nearest words


def min_dist_word(token, beamwidth = 10 ):
    mydict = {}
    for i in range(0,beamwidth):
        mydict[i] = 500 

    for word in vocabulary:
        d = levenshtein(token,word)
        key_max = max(mydict.keys(), key=(lambda k: mydict[k]))
        if d < mydict[key_max]:
            mydict.pop(key_max)
            mydict[word] = d

    for key in mydict:
        mydict[key] = 1/(mydict[key] + 1)

    return mydict # returns for each token, one dictionary with { word : 1/(dist +1) }




                
# Spelling corrector

def findWriteSentence(sentence,beamwidth=10):
    l1 = 0.4;
    l2 = 0.6;
    sentence = '<s> '+sentence; 
    sentence_array = tweet_wt.tokenize(sentence)
    
    editdistance = [] # table with dictionaries with the 10 closest words of each word of the sequence
    edtdistance = [min_dist_word(sentence_array[k],5) for k in range(1,len(sentence_array))]

    path = [{('<s>','<s>'):0}]
   
    for i in range(len(edtdistance)): 
        diction = dict()
        for (prevword, token) in path[len(path)-1]:
            for word in edtdistance[i]:
                LK = -l1* findBigramPropability(token, word) -l2* math.log2(edtdistance[i][word])  
                diction[token, word] = path[len(path)-1][prevword,token] +LK
        j = len(diction)
        
        while j >2:
            key_max = max(diction.keys(), key=(lambda k: diction[k]))
            diction.pop(key_max)
            j = j-1
        path.append(diction) 

    lastitem = path[len(path)-1];
    key_min = min(lastitem.keys(), key=(lambda k: lastitem[k]))

    prevword,lastword = key_min[0],key_min[1]
    right_sentence = lastword

    index= len(path)-2
    while(index > 0):
        previtem = path[index]
        for key in previtem:
            if (key[1] ==prevword):
                right_sentence = key[1]+' '+right_sentence

                prevword = key[0]
        index =index-1
    
    return(right_sentence)

initial_sentence = input('Give a sentence') #test with input sentence
print(findWriteSentence(initial_sentence))

